In [78]:
import os.path
import requests
import pandas as pd
import textwrap

def generate_sparql_query(fullName, property_labels_to_ids, language='en', qid=None):
    """
    Query WikiData for the properties of the given person listed in the given property map,
    either by fullName or QID. When a QID is provided, ?itemLabel is not included in the query.
    :param fullName: Name of the person to query
    :param property_labels_to_ids: Dictionary mapping property labels to WikiData property IDs
    :param language: Language code for the query results
    :param qid: WikiData entity ID (QID) for the person
    :return: SPARQL query string
    """

    if qid:
        selectClause = "SELECT DISTINCT ?item"
        itemConstraint = f"BIND(wd:{qid} AS ?item)."
        groupByClause = "GROUP BY ?item"
    else:
        selectClause = "SELECT DISTINCT ?item ?itemLabel"
        itemConstraint = f'?item wdt:P31 wd:Q5; rdfs:label "{fullName}"@{language} .'
        groupByClause = "GROUP BY ?item ?itemLabel"

    for label, pid in property_labels_to_ids.items():
        # add to property selection
        if label.endswith('Name'):
            selectClause += f'''
                (GROUP_CONCAT(DISTINCT ?{label}Value; separator=" ") AS ?{label})'''
        else:
            selectClause += f'''
                (SAMPLE(?{label}) AS ?{label})'''

        # add to item constraint
        if label.endswith("_id") or label.startswith("image") or label.startswith("date"):
            itemConstraint += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}. }}"""
        elif label.endswith("Name"):
            itemConstraint += f"""
                OPTIONAL {{
                ?item p:{pid} ?{label}Statement.
                    ?{label}Statement ps:{pid} ?{label}.
                    OPTIONAL {{ ?{label}Statement pq:P1545 ?order. }}
                    OPTIONAL {{
                ?{label} rdfs:label ?{label}Label.
                      FILTER(LANG(?{label}Label) = "{language}")
                    }}
                    BIND(COALESCE(?{label}Label, STR(?{label})) AS ?{label}Value)
                  }}"""
        else:
            itemConstraint += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}Id . ?{label}Id rdfs:label ?{label} FILTER(LANG(?{label}) = "{language}") . }}"""
    
    query = textwrap.dedent(f"""
    {selectClause} 
    WHERE {{
        {itemConstraint}
    }}
    {groupByClause}
    """)

    return query


def query_wikidata(fullName, property_map, language='en', qid=None, debug=False):
    SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"
    query = generate_sparql_query(fullName, property_map, language, qid=qid)
    if debug:
        print(query)
    headers = {'User-Agent': 'Mozilla/5.0', 'Accept': 'application/json'}
    response = requests.get(SPARQL_ENDPOINT, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code != 200:
        print(query)
        response.raise_for_status()
        
    results = response.json()['results']['bindings']
    
    if not results:
        return []

    for i, result in enumerate(results):
        # Initialize with fullName to ensure it appears first
        data = {
            'fullName': fullName
        }
   
        for label in property_map:
            if label in result:
                value = result[label]['value']
                data[label] = value
            else:
                data[label] = None
                
        # add qid and item URI
        data['qid'] = os.path.basename(result['item']['value'])
        data['wikidata_url'] = result['item']['value']
        results[i] = data
            
    return results

def get_wikipedia_links(qid, languages):
    """
    Fetch Wikipedia links for a given Wikidata QID and a list of languages.

    Parameters:
    - qid (str): The QID of the Wikidata item.
    - languages (list): A list of language codes (e.g., ['en', 'de']). 

    Returns:
    - dict: A dictionary with languages as keys and Wikipedia URLs as values.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "sitelinks/urls",
        "format": "json"
    }

    response = requests.get(url, params=params)
    data = response.json()
    links = {}
    if "entities" in data and qid in data["entities"]:
        sitelinks = data["entities"][qid].get("sitelinks", {})
        for lang in languages:
            sitekey = f"{lang}wiki"
            if sitekey in sitelinks:
                siteLinkData = sitelinks.get(sitekey)
                if 'url' in siteLinkData:
                    links[lang] = siteLinkData.get('url')
                else:
                    # Use the 'title' key and construct the URL manually
                    title = sitelinks[sitekey]["title"]
                    links[lang] = f"https://{lang}.wikipedia.org/wiki/{requests.utils.quote(title)}"
            else:
                links[lang] = None  # Or use '' to represent absence of link

    return links


def get_person_info_from_wikidata(names, property_map, languages=None, debug=False):
    if languages is None:
        languages = ['en', 'de']
    all_data = []
    print('Retrieving scholar data...')
    for item in names:
        if type(item) is tuple:
            results = query_wikidata(item[0], property_map, languages[0], qid=item[1], debug=debug)
        else:
            results = query_wikidata(item, property_map, languages[0], debug=debug)
        
        all_data += results

    # Ensure fullName appears first by reordering columns based on property_labels_to_ids keys
    columns = ['fullName', 'qid'] + list(property_map.keys()) + ['wikidata_url'] + [f'wikipedia_{l}' for l in languages]
    
    if len(all_data) > 0:
        df = pd.DataFrame(all_data, columns=columns, dtype=str)
        # Add wikipedia links
        print("Retrieving wikipedia URLs...")
        # For each QID in the DataFrame, fetch Wikipedia links for all languages and update the DataFrame accordingly
        for index, row in df.iterrows():
            qid = row['qid']
            links = get_wikipedia_links(qid, languages)
        
            # Update the DataFrame directly with the fetched links for each language
            for language in languages:
                df.at[index, f'wikipedia_{language}'] = links.get(language, None)
    else:
        df = pd.DataFrame(columns=columns, dtype=str)
    return df

## Retrieve Data for a given list of scholars

In [79]:
# Now calling the updated function with the 'language' parameter
property_labels_to_ids = {
    'sexOrGender': 'P21',
#    'image': 'P18',
#    'countryOfCitizenship': 'P27',
    'familyName': 'P734',
    'givenName': 'P735',
    'dateOfBirth': 'P569',
    'dateOfDeath': 'P570',
#    'occupation': 'P106',
#    'fieldOfWork': 'P101',
#    'viaf_id': 'P214',
#    'isni_id': 'P213',
    'gnd_id': 'P227'
}

scholars = [
    "Hans Kelsen",
    "Hugo Sinzheimer",
    ("Karl Renner","Q11726"),
    ("Ernst Fraenkel", "Q86812"),
    ("Franz Leopold Neumann", "Q63195"),
    "Otto Kahn-Freund",
    "Otto Kirchheimer",
    "Herrmann Kantorowicz",
    ("Ludwig Bendix", "Q15449424"),
    ("Arthur Nussbaum", "Q103088"),
    "Theodor Geiger",
    "Erhard Blankenburg",
    "Wolfgang Kaupen",
    "Rüdiger Lautmann",
    "Thilo Ramm",
    "Rudolf Wiethölter",
    ("Niklas Luhmann","Q57238"),
    'Hubert Rottleuthner',
    'Ralf Rogowski',
    "Gunther Teubner",
    "Volkmar Gessner",
    "Konstanze Plett",
    "Ute Sacksofsky",
    ("Susanne Baer","Q101872")
]
df = get_person_info_from_wikidata(scholars, property_labels_to_ids, debug=False)
# Convert date strings to datetime
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth'])
df['dateOfDeath'] = pd.to_datetime(df['dateOfDeath'])
df.to_csv("scholars.csv", index=False)
df

Retrieving scholar data...
Retrieving wikipedia URLs...


,fullName,qid,sexOrGender,familyName,givenName,dateOfBirth,dateOfDeath,gnd_id,wikidata_url,wikipedia_en,wikipedia_de
0,Hans Kelsen,Q84165,male,Kelsen,Hans,1881-10-11 00:00:00+00:00,1973-04-19 00:00:00+00:00,118561219,http://www.wikidata.org/entity/Q84165,https://en.wikipedia.org/wiki/Hans_Kelsen,https://de.wikipedia.org/wiki/Hans_Kelsen
1,Hugo Sinzheimer,Q86043,male,Sinzheimer,Hugo D.,1875-01-01 00:00:00+00:00,1945-09-16 00:00:00+00:00,118614711,http://www.wikidata.org/entity/Q86043,https://en.wikipedia.org/wiki/Hugo_Sinzheimer,https://de.wikipedia.org/wiki/Hugo_Sinzheimer
2,Karl Renner,Q11726,male,Renner,Karl,1870-12-14 00:00:00+00:00,1950-12-31 00:00:00+00:00,118599739,http://www.wikidata.org/entity/Q11726,https://en.wikipedia.org/wiki/Karl_Renner,https://de.wikipedia.org/wiki/Karl_Renner
3,Ernst Fraenkel,Q86812,male,Fraenkel,Ernst,1898-12-26 00:00:00+00:00,1975-03-28 00:00:00+00:00,118534602,http://www.wikidata.org/entity/Q86812,https://en.wikipedia.org/wiki/Ernst_Fraenkel_(...,https://de.wikipedia.org/wiki/Ernst_Fraenkel_(...
4,Franz Leopold Neumann,Q63195,male,Neumann,Leopold Franz,1900-05-23 00:00:00+00:00,1954-09-02 00:00:00+00:00,118587293,http://www.wikidata.org/entity/Q63195,https://en.wikipedia.org/wiki/Franz_Neumann_(p...,https://de.wikipedia.org/wiki/Franz_Neumann_(P...
5,Otto Kahn-Freund,Q121832,male,Kahn Freund,Otto,1900-11-17 00:00:00+00:00,1979-06-16 00:00:00+00:00,118559362,http://www.wikidata.org/entity/Q121832,https://en.wikipedia.org/wiki/Otto_Kahn-Freund,https://de.wikipedia.org/wiki/Otto_Kahn-Freund
6,Otto Kirchheimer,Q214397,male,Kirchheimer,Otto,1905-11-11 00:00:00+00:00,1965-11-22 00:00:00+00:00,118562371,http://www.wikidata.org/entity/Q214397,https://en.wikipedia.org/wiki/Otto_Kirchheimer,https://de.wikipedia.org/wiki/Otto_Kirchheimer
7,Ludwig Bendix,Q15449424,male,Bendix,Ludwig,1877-06-28 00:00:00+00:00,1954-01-03 00:00:00+00:00,118702033,http://www.wikidata.org/entity/Q15449424,None,https://de.wikipedia.org/wiki/Ludwig_Bendix
8,Arthur Nussbaum,Q103088,male,Nussbaum,Arthur,1877-01-01 00:00:00+00:00,1964-01-01 00:00:00+00:00,117071676,http://www.wikidata.org/entity/Q103088,https://en.wikipedia.org/wiki/Arthur_Nussbaum,https://de.wikipedia.org/wiki/Arthur_Nussbaum
9,Theodor Geiger,Q96410,male,Geiger,Theodor,1891-11-09 00:00:00+00:00,1952-06-16 00:00:00+00:00,118538187,http://www.wikidata.org/entity/Q96410,https://en.wikipedia.org/wiki/Theodor_Geiger,https://de.wikipedia.org/wiki/Theodor_Geiger


## Create a list of scholar's QIDs only

In [83]:
df.copy()[['fullName', 'qid']].to_csv("scholars-qid.csv", index=False)

## Create a clickable list of Wikidata/Wikipedia URLs

In [76]:
import pandas as pd
from IPython.display import display, Markdown

# Function to convert URLs to HTML links
def make_clickable(val, name):
    return f'<a target="_blank" href="{val}">{name}</a>'

# Apply the function to each URL column
df_styled = df.copy()[['fullName', 'qid', 'familyName', 'givenName', 'wikidata_url', 'wikipedia_en', 'wikipedia_de' ]]
df_styled['wikidata_url'] = df['wikidata_url'].apply(make_clickable, name='Wikidata')
df_styled['wikipedia_en'] = df['wikipedia_en'].apply(make_clickable, name='Wikipedia (EN)')
df_styled['wikipedia_de'] = df['wikipedia_de'].apply(make_clickable, name='Wikipedia (DE)')

# Display the DataFrame as HTML
display(Markdown(df_styled.to_markdown()))


|    | fullName              | qid       | familyName   | givenName     | wikidata_url                                                                    | wikipedia_en                                                                                                    | wikipedia_de                                                                                                       |
|---:|:----------------------|:----------|:-------------|:--------------|:--------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------|
|  0 | Hans Kelsen           | Q84165    | Kelsen       | Hans          | <a target="_blank" href="http://www.wikidata.org/entity/Q84165">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Hans_Kelsen">Wikipedia (EN)</a>                          | <a target="_blank" href="https://de.wikipedia.org/wiki/Hans_Kelsen">Wikipedia (DE)</a>                             |
|  1 | Hugo Sinzheimer       | Q86043    | Sinzheimer   | Hugo D.       | <a target="_blank" href="http://www.wikidata.org/entity/Q86043">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Hugo_Sinzheimer">Wikipedia (EN)</a>                      | <a target="_blank" href="https://de.wikipedia.org/wiki/Hugo_Sinzheimer">Wikipedia (DE)</a>                         |
|  2 | Karl Renner           | Q11726    | Renner       | Karl          | <a target="_blank" href="http://www.wikidata.org/entity/Q11726">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Karl_Renner">Wikipedia (EN)</a>                          | <a target="_blank" href="https://de.wikipedia.org/wiki/Karl_Renner">Wikipedia (DE)</a>                             |
|  3 | Ernst Fraenkel        | Q86812    | Fraenkel     | Ernst         | <a target="_blank" href="http://www.wikidata.org/entity/Q86812">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Ernst_Fraenkel_(political_scientist)">Wikipedia (EN)</a> | <a target="_blank" href="https://de.wikipedia.org/wiki/Ernst_Fraenkel_(Politikwissenschaftler)">Wikipedia (DE)</a> |
|  4 | Franz Leopold Neumann | Q63195    | Neumann      | Leopold Franz | <a target="_blank" href="http://www.wikidata.org/entity/Q63195">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Franz_Neumann_(political_scientist)">Wikipedia (EN)</a>  | <a target="_blank" href="https://de.wikipedia.org/wiki/Franz_Neumann_(Politikwissenschaftler)">Wikipedia (DE)</a>  |
|  5 | Otto Kahn-Freund      | Q121832   | Kahn Freund  | Otto          | <a target="_blank" href="http://www.wikidata.org/entity/Q121832">Wikidata</a>   | <a target="_blank" href="https://en.wikipedia.org/wiki/Otto_Kahn-Freund">Wikipedia (EN)</a>                     | <a target="_blank" href="https://de.wikipedia.org/wiki/Otto_Kahn-Freund">Wikipedia (DE)</a>                        |
|  6 | Otto Kirchheimer      | Q214397   | Kirchheimer  | Otto          | <a target="_blank" href="http://www.wikidata.org/entity/Q214397">Wikidata</a>   | <a target="_blank" href="https://en.wikipedia.org/wiki/Otto_Kirchheimer">Wikipedia (EN)</a>                     | <a target="_blank" href="https://de.wikipedia.org/wiki/Otto_Kirchheimer">Wikipedia (DE)</a>                        |
|  7 | Ludwig Bendix         | Q15449424 | Bendix       | Ludwig        | <a target="_blank" href="http://www.wikidata.org/entity/Q15449424">Wikidata</a> | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="https://de.wikipedia.org/wiki/Ludwig_Bendix">Wikipedia (DE)</a>                           |
|  8 | Arthur Nussbaum       | Q103088   | Nussbaum     | Arthur        | <a target="_blank" href="http://www.wikidata.org/entity/Q103088">Wikidata</a>   | <a target="_blank" href="https://en.wikipedia.org/wiki/Arthur_Nussbaum">Wikipedia (EN)</a>                      | <a target="_blank" href="https://de.wikipedia.org/wiki/Arthur_Nussbaum">Wikipedia (DE)</a>                         |
|  9 | Theodor Geiger        | Q96410    | Geiger       | Theodor       | <a target="_blank" href="http://www.wikidata.org/entity/Q96410">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Theodor_Geiger">Wikipedia (EN)</a>                       | <a target="_blank" href="https://de.wikipedia.org/wiki/Theodor_Geiger">Wikipedia (DE)</a>                          |
| 10 | Erhard Blankenburg    | Q51595283 | Blankenburg  | Erhard        | <a target="_blank" href="http://www.wikidata.org/entity/Q51595283">Wikidata</a> | <a target="_blank" href="https://en.wikipedia.org/wiki/Erhard_Blankenburg">Wikipedia (EN)</a>                   | <a target="_blank" href="https://de.wikipedia.org/wiki/Erhard_Blankenburg">Wikipedia (DE)</a>                      |
| 11 | Wolfgang Kaupen       | Q93221485 |              | Wolfgang      | <a target="_blank" href="http://www.wikidata.org/entity/Q93221485">Wikidata</a> | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="None">Wikipedia (DE)</a>                                                                  |
| 12 | Rüdiger Lautmann      | Q91074    |              | Rüdiger       | <a target="_blank" href="http://www.wikidata.org/entity/Q91074">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/R%C3%BCdiger_Lautmann">Wikipedia (EN)</a>                | <a target="_blank" href="https://de.wikipedia.org/wiki/R%C3%BCdiger_Lautmann">Wikipedia (DE)</a>                   |
| 13 | Thilo Ramm            | Q59533838 | Ramm         | Thilo         | <a target="_blank" href="http://www.wikidata.org/entity/Q59533838">Wikidata</a> | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="https://de.wikipedia.org/wiki/Thilo_Ramm">Wikipedia (DE)</a>                              |
| 14 | Rudolf Wiethölter     | Q1512482  |              | Rudolf        | <a target="_blank" href="http://www.wikidata.org/entity/Q1512482">Wikidata</a>  | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="https://de.wikipedia.org/wiki/Rudolf_Wieth%C3%B6lter">Wikipedia (DE)</a>                  |
| 15 | Niklas Luhmann        | Q57238    | Luhmann      | Niklas        | <a target="_blank" href="http://www.wikidata.org/entity/Q57238">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Niklas_Luhmann">Wikipedia (EN)</a>                       | <a target="_blank" href="https://de.wikipedia.org/wiki/Niklas_Luhmann">Wikipedia (DE)</a>                          |
| 16 | Hubert Rottleuthner   | Q55622018 |              | Hubert        | <a target="_blank" href="http://www.wikidata.org/entity/Q55622018">Wikidata</a> | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="https://de.wikipedia.org/wiki/Hubert_Rottleuthner">Wikipedia (DE)</a>                     |
| 17 | Gunther Teubner       | Q98304    | Teubner      | Gunther       | <a target="_blank" href="http://www.wikidata.org/entity/Q98304">Wikidata</a>    | <a target="_blank" href="https://en.wikipedia.org/wiki/Gunther_Teubner">Wikipedia (EN)</a>                      | <a target="_blank" href="https://de.wikipedia.org/wiki/Gunther_Teubner">Wikipedia (DE)</a>                         |
| 18 | Volkmar Gessner       | Q15435946 | Gessner      | Volkmar       | <a target="_blank" href="http://www.wikidata.org/entity/Q15435946">Wikidata</a> | <a target="_blank" href="https://en.wikipedia.org/wiki/Volkmar_Gessner">Wikipedia (EN)</a>                      | <a target="_blank" href="https://de.wikipedia.org/wiki/Volkmar_Gessner">Wikipedia (DE)</a>                         |
| 19 | Konstanze Plett       | Q95192683 |              |               | <a target="_blank" href="http://www.wikidata.org/entity/Q95192683">Wikidata</a> | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="https://de.wikipedia.org/wiki/Konstanze_Plett">Wikipedia (DE)</a>                         |
| 20 | Ute Sacksofsky        | Q48562036 |              | Ute           | <a target="_blank" href="http://www.wikidata.org/entity/Q48562036">Wikidata</a> | <a target="_blank" href="None">Wikipedia (EN)</a>                                                               | <a target="_blank" href="https://de.wikipedia.org/wiki/Ute_Sacksofsky">Wikipedia (DE)</a>                          |
| 21 | Susanne Baer          | Q101872   | Baer         | Susanne       | <a target="_blank" href="http://www.wikidata.org/entity/Q101872">Wikidata</a>   | <a target="_blank" href="https://en.wikipedia.org/wiki/Susanne_Baer">Wikipedia (EN)</a>                         | <a target="_blank" href="https://de.wikipedia.org/wiki/Susanne_Baer">Wikipedia (DE)</a>                            |

In [13]:
import pandas as pd
import plotly.express as px

# Load the data
df = pd.read_csv("scholars.csv", encoding='utf-8')

# Initialize a list to track the last dateOfDeath in each row to manage overlaps
last_dates = []

# Function to find the appropriate row for each scholar
def find_row(last_dates, start_date):
    for i, last_date in enumerate(last_dates):
        if start_date > last_date:
            return i
    return len(last_dates)

# Assign rows without overlaps and sort by the earliest dateOfBirth
df['row'] = 0
for index, scholar in df.iterrows():
    row = find_row(last_dates, scholar['dateOfBirth'])
    if row < len(last_dates):
        last_dates[row] = scholar['dateOfDeath']
    else:
        last_dates.append(scholar['dateOfDeath'])
    df.at[index, 'row'] = row

# Now plotting without row labels
fig = px.timeline(df, x_start="dateOfBirth", x_end="dateOfDeath", y="row", text="fullName", title="Scholars' Life Spans Timeline")

# Update layout
fig.update_layout(yaxis=dict(tickmode='array', tickvals=[], ticktext=[]))
fig.update_yaxes(autorange="reversed")  # This reverses the y-axis to match your requirement

fig.show()



TypeError: '>' not supported between instances of 'float' and 'str'

In [44]:
import pandas as pd
from datetime import datetime

# Assuming df is your existing DataFrame

# Convert dateOfBirth and dateOfDeath to just the year, handle NaT/NaN appropriately
df['Year'] = pd.to_datetime(df['dateOfBirth'], errors='coerce').dt.year.astype('Int64')
df['End Year'] = pd.to_datetime(df['dateOfDeath'], errors='coerce').dt.year.astype('Int64')

# Create 'Display Date' as "dateOfBirth - dateOfDeath"
df['Display Date'] = df['Year'].astype(str).replace('<NA>','')  + ' - ' + df['End Year'].astype(str).replace('<NA>','')

# Create 'Headline' as "fullName (dateOfBirth - dateOfDeath)"
df['Headline'] = df['fullName'] + ' (' + df['Display Date'] + ')'

# Create 'Text' column by combining occupation, fieldOfWork, employer
df['Text'] = df[['occupation', 'fieldOfWork']].apply(lambda x: '<br>'.join(x.dropna()), axis=1)

# Use the image directly; assuming the URLs are already correctly formed in the 'image' column
df['Media'] = df['image']

# Add a "Group" column with the value "actors" for all rows
df['Group'] = 'actors'

# fix date columns
df['Display Date'] = df['Display Date'].fillna('')  # Ensure no NaNs in Display Date
df['Headline'] = df['Headline'].fillna('')  # Ensure no NaNs in Headline
df['Text'] = df['Text'].fillna('')  # Ensure no NaNs in Text
df['Media'] = df['Media'].fillna('')  # Ensure no NaNs in Media

# Now select and order the DataFrame according to the TimelineJS template requirements
columns = "Year	Month	Day	Time	End Year	End Month	End Day	End Time	Display Date	Headline	Text	Media	Media Credit	Media Caption	Media Thumbnail	Type	Group	Background	Link".split("\t")
for col in columns:
    if col not in df:
        df[col] = ''
timeline_df = df[columns]

timeline_df.to_excel("timeline_data.xlsx", index=False)
